### Idea Working Towards:
- Load well with LASIO
- Export LASIO style JSON
- CONVERT LASIO style JSON to WELLIO style JSON
- Visualize with wellioviz

In [ ]:
import json

In [ ]:
#!pip install matplotlib
##!pip install pixiedust_node

ABOUT PIXIEDUST NODE
WHICH LETS YOU RUN NODE.js CODE IN JUPYTER
https://github.com/pixiedust/pixiedust_node

In [ ]:
import pixiedust_node

In [ ]:
%%node
//// THIS IS A JavaScript CELL
var str1 = "hello world";
var n1 = 4.1515;
var n2 = 42;
var tf = true;
var obj = { name:"Frank", age: 42 };
var array_of_strings = ["hello", "world"];
var array_of_objects = [{a:1,b:2}, {a:3, b:4}];

In [ ]:
#### THIS IS A PYTHON CELL
print(str1, n1, n2, tf)
print(obj)
print(array_of_strings)
print(array_of_objects)

Geting wellio.js

In [6]:
npm.install('wellio')

/usr/local/bin/npm install -s wellio
+ wellio@0.1.7
updated 1 package and audited 313 packages in 0.69s
found 0 vulnerabilities


In [7]:
%%node
var wellio = require('wellio');

test of wellio

In [8]:
%%node
console.log(wellio.returnThing("asdfa"))

asdfa


Now lets get lasio

In [9]:
#!pip install lasio

In [10]:
import lasio

In [11]:
import numpy as np
from datetime import datetime

In [12]:
datetime.today().strftime('%Y-%m-%d %H:%M:%S')

'2020-03-26 15:37:56'

In [13]:
las = lasio.LASFile()

las.well.DATE = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
las.params['ENG'] = lasio.HeaderItem('ENG', value='Kent Inverarity')
las.params['LMF'] = lasio.HeaderItem('LMF', value='GL')
las.other = 'Example of how to create a LAS file from scratch using lasio'
depths = np.arange(10, 50, 0.5)
synth = np.log10(depths)*5+np.random.random(len(depths))
synth[:8] = np.nan
las.add_curve('DEPT', depths, unit='m')
las.add_curve('SYNTH', synth, descr='fake data')
las.write('scratch_v1.2.las', version=1.2)
las.write('scratch_v2.las', version=2)

In [14]:
las

In [15]:
las.curves

[CurveItem(mnemonic=DEPT, unit=m, value=, descr=, original_mnemonic=DEPT, data.shape=(80,)),
 CurveItem(mnemonic=SYNTH, unit=, value=, descr=fake data, original_mnemonic=SYNTH, data.shape=(80,))]

In [16]:
json_images = json.dumps(las, cls=lasio.JSONEncoder)

In [17]:
json_images

'{"metadata": {"Version": [{}, {}, {}], "Well": [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], "Curves": [{}, {}], "Parameter": [{}, {}], "Other": "Example of how to create a LAS file from scratch using lasio"}, "data": {"DEPT": [10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0, 28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5, 35.0, 35.5, 36.0, 36.5, 37.0, 37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5], "SYNTH": [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 6.587021099949073, 5.8381427419239715, 6.848492979587442, 6.640354973079276, 6.786804986640453, 7.0029528863890125, 6.404811954951237, 6.801948659987526, 6.420852191627129, 6.969307216130774, 7.161771364907318, 7.2481282

In [18]:
las_json_dict =json.loads(json_images)

In [19]:
with open('data-org.json', 'w') as outfile:
    json.dump(las_json_dict, outfile)

In [20]:
!ls

Wellio Demo in Jupyter Notebook Node.js.ipynb mytmp.json                                    tmp.js
bad.ipynb                                     requirements.txt                              wellio.js and lasio test.ipynb
data-org.json                                 scratch_v1.2.las                              wl.ipynb
data.json                                     scratch_v2.las


In [21]:
%%node
const path = require('path');
let mydir = process.env.PWD;
let myfile = mydir + path.sep + 'data.json';

let lasio_json_str = '';
let lasio_obj = '';
let wellio_obj = '';

try {
    lasio_json_str = wellio.read_lasio_json_file(myfile);
    lasio_obj = JSON.parse(lasio_json_str);
    wellio_obj = wellio.lasio_obj_2_wellio_obj(lasio_obj);
} catch (e) {
    console.log('[');
    console.log(e.name + ":: " + e.message);
    console.log(']');
}


console.log(wellio_obj);

lasjson = wellio_obj;

// wellio.read_json('data.json')

... ... ... ... ... ... ... ...
{
'VERSION INFORMATION': {
VERS: {
MNEM: 'VERS',
UNIT: '',
DATA: 2,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
WRAP: {
MNEM: 'WRAP',
UNIT: '',
DATA: 'NO',
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
DLM: {
MNEM: 'DLM',
UNIT: '',
DATA: 'SPACE',
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
}
},
'WELL INFORMATION BLOCK': {
STRT: {
MNEM: 'STRT',
UNIT: '',
DATA: null,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
STOP: {
MNEM: 'STOP',
UNIT: '',
DATA: null,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
STEP: {
MNEM: 'STEP',
UNIT: '',
DATA: null,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
NULL: {
MNEM: 'NULL',
UNIT: '',
DATA: -9999.25,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
COMP: {
MNEM: 'COMP',
UNIT: '',
DATA: '',
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
WELL: {
MNEM: '

#### THE fs to upload the json file doesn't work well for some reason in this setting, 
#### so switched to just grabbing the a json dict I made from the lassio json in string form
#### I still used "most" of the read_json function

In [22]:
%%node

/*
    var lj3 = las_json_dict
    std_headers = {
      'Version': 'VERSION INFORMATION',
      'Well': 'WELL INFORMATION BLOCK',
      'Curves': 'CURVE INFORMATION BLOCK', 
      'Parameter': 'PARAMETER INFORMATION'
    };

    lasjson = {};
    lasjson["VERSION INFORMATION"] = {};
    lasjson["WELL INFORMATION BLOCK"] = {};
    lasjson["CURVE INFORMATION BLOCK"] = {};
    lasjson["PARAMETER INFORMATION"] = {};
    lasjson["CURVES"] = lj3.data;

    for (item in lj3.metadata) {
      if ( !(item in std_headers) ) {
        lasjson[item.toUpperCase()] = lj3.metadata[item];
      }
      else {
        for (var mnemonic in lj3.metadata[item]) {
          section = std_headers[item];      
          lasjson[section][mnemonic] = {
            MNEM: mnemonic,
            UNIT: '',
            DATA: lj3.metadata[item][mnemonic],
            'DESCRIPTION OF MNEMONIC 1': '',
            'DESCRIPTION OF MNEMONIC 2': ''
          };
        }
      }
    }
    */

... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ..... ..... ... ..... ....... ....... ......... ......... ......... ......... ......... ......... ....... ..... ... ...


In [23]:
%%node
console.log(lasjson)

{
'VERSION INFORMATION': {
VERS: {
MNEM: 'VERS',
UNIT: '',
DATA: 2,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
WRAP: {
MNEM: 'WRAP',
UNIT: '',
DATA: 'NO',
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
DLM: {
MNEM: 'DLM',
UNIT: '',
DATA: 'SPACE',
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
}
},
'WELL INFORMATION BLOCK': {
STRT: {
MNEM: 'STRT',
UNIT: '',
DATA: null,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
STOP: {
MNEM: 'STOP',
UNIT: '',
DATA: null,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
STEP: {
MNEM: 'STEP',
UNIT: '',
DATA: null,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
NULL: {
MNEM: 'NULL',
UNIT: '',
DATA: -9999.25,
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
COMP: {
MNEM: 'COMP',
UNIT: '',
DATA: '',
'DESCRIPTION OF MNEMONIC 1': '',
'DESCRIPTION OF MNEMONIC 2': ''
},
WELL: {
MNEM: 'WELL',
UNIT: '',
DATA: '',
'DESC

### and it works!

## We should probably write two read LASIO json functions to separate the file system upload from the one that parses the dict... That way people can either (1) upload from JSON file in lasio style of JSON (2) start from lasio style json string (3) start from lasio style dict

## Now let's try to visualize that JSON with wellioviz

In [24]:
lasjson

{'VERSION INFORMATION': {'VERS': {'MNEM': 'VERS',
   'UNIT': '',
   'DATA': 2,
   'DESCRIPTION OF MNEMONIC 1': '',
   'DESCRIPTION OF MNEMONIC 2': ''},
  'WRAP': {'MNEM': 'WRAP',
   'UNIT': '',
   'DATA': 'NO',
   'DESCRIPTION OF MNEMONIC 1': '',
   'DESCRIPTION OF MNEMONIC 2': ''},
  'DLM': {'MNEM': 'DLM',
   'UNIT': '',
   'DATA': 'SPACE',
   'DESCRIPTION OF MNEMONIC 1': '',
   'DESCRIPTION OF MNEMONIC 2': ''}},
 'WELL INFORMATION BLOCK': {'STRT': {'MNEM': 'STRT',
   'UNIT': '',
   'DATA': None,
   'DESCRIPTION OF MNEMONIC 1': '',
   'DESCRIPTION OF MNEMONIC 2': ''},
  'STOP': {'MNEM': 'STOP',
   'UNIT': '',
   'DATA': None,
   'DESCRIPTION OF MNEMONIC 1': '',
   'DESCRIPTION OF MNEMONIC 2': ''},
  'STEP': {'MNEM': 'STEP',
   'UNIT': '',
   'DATA': None,
   'DESCRIPTION OF MNEMONIC 1': '',
   'DESCRIPTION OF MNEMONIC 2': ''},
  'NULL': {'MNEM': 'NULL',
   'UNIT': '',
   'DATA': -9999.25,
   'DESCRIPTION OF MNEMONIC 1': '',
   'DESCRIPTION OF MNEMONIC 2': ''},
  'COMP': {'MNEM': 'COMP

First, lets bring in wellioviz

In [25]:
npm.install('wellioviz')

/usr/local/bin/npm install -s wellioviz
+ wellioviz@0.0.18
updated 1 package and audited 313 packages in 0.476s
found 0 vulnerabilities


In [26]:
%%node 
wellioviz = require('wellioviz')

In [27]:
%%node
console.log(wellioviz.define_wellioviz())

WELLIOVIZ is a JavaScript library that provides functionality to visualize well logs, particularly those already converted to JSON, using d3.js visualization library.


In [28]:
%%node
three_things_2 = wellioviz.fromJSONofWEllGetThingsForPlotting(lasjson,"DEPT")

hmm... might be that the fake LAS minimal LAS file is too minimal?
Let's try a real one:

In [29]:
las2 = lasio.read("../assets/00-01-03-085-15W4-0.LAS")

In [30]:
las2

In [31]:
las2_json_string = las2.json

In [32]:
las2_json_string

'{"Version": "[\\"{\\\\\\"_type\\\\\\": \\\\\\"HeaderItem\\\\\\", \\\\\\"mnemonic\\\\\\": \\\\\\"VERS\\\\\\", \\\\\\"unit\\\\\\": \\\\\\"\\\\\\", \\\\\\"value\\\\\\": 2.0, \\\\\\"descr\\\\\\": \\\\\\"CWLS LOG ASCII STANDARD -VERSION 2.0\\\\\\"}\\", \\"{\\\\\\"_type\\\\\\": \\\\\\"HeaderItem\\\\\\", \\\\\\"mnemonic\\\\\\": \\\\\\"WRAP\\\\\\", \\\\\\"unit\\\\\\": \\\\\\"\\\\\\", \\\\\\"value\\\\\\": \\\\\\"NO\\\\\\", \\\\\\"descr\\\\\\": \\\\\\"ONE LINE PER DEPTH STEP\\\\\\"}\\"]", "Well": "[\\"{\\\\\\"_type\\\\\\": \\\\\\"HeaderItem\\\\\\", \\\\\\"mnemonic\\\\\\": \\\\\\"NULL\\\\\\", \\\\\\"unit\\\\\\": \\\\\\"\\\\\\", \\\\\\"value\\\\\\": -999.25, \\\\\\"descr\\\\\\": \\\\\\"NULL_VALUE\\\\\\"}\\", \\"{\\\\\\"_type\\\\\\": \\\\\\"HeaderItem\\\\\\", \\\\\\"mnemonic\\\\\\": \\\\\\"WELL\\\\\\", \\\\\\"unit\\\\\\": \\\\\\"\\\\\\", \\\\\\"value\\\\\\": \\\\\\"PETROREP ET AL HOUSE\\\\\\", \\\\\\"descr\\\\\\": \\\\\\"Well_name    - WELL\\\\\\"}\\", \\"{\\\\\\"_type\\\\\\": \\\\\\"HeaderItem\\\

In [33]:
las2_json_string_cleaned = las2_json_string.replace("\\\\\\","").replace("     ","").replace('\\"',"").replace('"\\',"").replace('"]',"]").replace(']"',"]").replace('["',"[").replace('"[',"[")


In [34]:
las2_json_string_cleaned

'{"Version": [{"_type": "HeaderItem", "mnemonic": "VERS", "unit": "", "value": 2.0, "descr": "CWLS LOG ASCII STANDARD -VERSION 2.0"}, {"_type": "HeaderItem", "mnemonic": "WRAP", "unit": "", "value": "NO", "descr": "ONE LINE PER DEPTH STEP"}], "Well": [{"_type": "HeaderItem", "mnemonic": "NULL", "unit": "", "value": -999.25, "descr": "NULL_VALUE"}, {"_type": "HeaderItem", "mnemonic": "WELL", "unit": "", "value": "PETROREP ET AL HOUSE", "descr": "Well_name    - WELL"}, {"_type": "HeaderItem", "mnemonic": "LOC", "unit": "", "value": "01-03-085-15W4", "descr": "Location- LOCATION"}, {"_type": "HeaderItem", "mnemonic": "UWI", "unit": "", "value": "00/01-03-085-15W400", "descr": "Uwi- UNIQUE WELL ID"}, {"_type": "HeaderItem", "mnemonic": "ENTR", "unit": "", "value": "DW", "descr": "Entered - ENTERED BY"}, {"_type": "HeaderItem", "mnemonic": "SRVC", "unit": "", "value": "COMPUTALOG", "descr": "Scn- SERVICE COMPANY"}, {"_type": "HeaderItem", "mnemonic": "DATE", "unit": "", "value": "11 JAN 85"

In [35]:
las2_json_dict_cleaned = json.loads(las2_json_string_cleaned)

In [36]:
las2_json_dict_cleaned

{'Version': [{'_type': 'HeaderItem',
   'mnemonic': 'VERS',
   'unit': '',
   'value': 2,
   'descr': 'CWLS LOG ASCII STANDARD -VERSION 2.0'},
  {'_type': 'HeaderItem',
   'mnemonic': 'WRAP',
   'unit': '',
   'value': 'NO',
   'descr': 'ONE LINE PER DEPTH STEP'}],
 'Well': [{'_type': 'HeaderItem',
   'mnemonic': 'NULL',
   'unit': '',
   'value': -999.25,
   'descr': 'NULL_VALUE'},
  {'_type': 'HeaderItem',
   'mnemonic': 'WELL',
   'unit': '',
   'value': 'PETROREP ET AL HOUSE',
   'descr': 'Well_name    - WELL'},
  {'_type': 'HeaderItem',
   'mnemonic': 'LOC',
   'unit': '',
   'value': '01-03-085-15W4',
   'descr': 'Location- LOCATION'},
  {'_type': 'HeaderItem',
   'mnemonic': 'UWI',
   'unit': '',
   'value': '00/01-03-085-15W400',
   'descr': 'Uwi- UNIQUE WELL ID'},
  {'_type': 'HeaderItem',
   'mnemonic': 'ENTR',
   'unit': '',
   'value': 'DW',
   'descr': 'Entered - ENTERED BY'},
  {'_type': 'HeaderItem',
   'mnemonic': 'SRVC',
   'unit': '',
   'value': 'COMPUTALOG',
   'des

In [37]:
las2_json_string_cleanedB = json.dumps(las2_json_string_cleaned, cls=lasio.JSONEncoder)
las2_json_dict_cleanedB =json.loads(las2_json_string_cleanedB)

In [38]:
las2_json_string_cleanedB

'"{\\"Version\\": [{\\"_type\\": \\"HeaderItem\\", \\"mnemonic\\": \\"VERS\\", \\"unit\\": \\"\\", \\"value\\": 2.0, \\"descr\\": \\"CWLS LOG ASCII STANDARD -VERSION 2.0\\"}, {\\"_type\\": \\"HeaderItem\\", \\"mnemonic\\": \\"WRAP\\", \\"unit\\": \\"\\", \\"value\\": \\"NO\\", \\"descr\\": \\"ONE LINE PER DEPTH STEP\\"}], \\"Well\\": [{\\"_type\\": \\"HeaderItem\\", \\"mnemonic\\": \\"NULL\\", \\"unit\\": \\"\\", \\"value\\": -999.25, \\"descr\\": \\"NULL_VALUE\\"}, {\\"_type\\": \\"HeaderItem\\", \\"mnemonic\\": \\"WELL\\", \\"unit\\": \\"\\", \\"value\\": \\"PETROREP ET AL HOUSE\\", \\"descr\\": \\"Well_name    - WELL\\"}, {\\"_type\\": \\"HeaderItem\\", \\"mnemonic\\": \\"LOC\\", \\"unit\\": \\"\\", \\"value\\": \\"01-03-085-15W4\\", \\"descr\\": \\"Location- LOCATION\\"}, {\\"_type\\": \\"HeaderItem\\", \\"mnemonic\\": \\"UWI\\", \\"unit\\": \\"\\", \\"value\\": \\"00/01-03-085-15W400\\", \\"descr\\": \\"Uwi- UNIQUE WELL ID\\"}, {\\"_type\\": \\"HeaderItem\\", \\"mnemonic\\": \\"EN

In [39]:
las2_json_string_cleanedB_2 = json.loads(las2_json_string_cleanedB)

In [40]:
type(las2_json_string_cleanedB_2)

str

In [41]:
eval(las2_json_string_cleanedB_2)

{'Version': [{'_type': 'HeaderItem',
   'mnemonic': 'VERS',
   'unit': '',
   'value': 2.0,
   'descr': 'CWLS LOG ASCII STANDARD -VERSION 2.0'},
  {'_type': 'HeaderItem',
   'mnemonic': 'WRAP',
   'unit': '',
   'value': 'NO',
   'descr': 'ONE LINE PER DEPTH STEP'}],
 'Well': [{'_type': 'HeaderItem',
   'mnemonic': 'NULL',
   'unit': '',
   'value': -999.25,
   'descr': 'NULL_VALUE'},
  {'_type': 'HeaderItem',
   'mnemonic': 'WELL',
   'unit': '',
   'value': 'PETROREP ET AL HOUSE',
   'descr': 'Well_name    - WELL'},
  {'_type': 'HeaderItem',
   'mnemonic': 'LOC',
   'unit': '',
   'value': '01-03-085-15W4',
   'descr': 'Location- LOCATION'},
  {'_type': 'HeaderItem',
   'mnemonic': 'UWI',
   'unit': '',
   'value': '00/01-03-085-15W400',
   'descr': 'Uwi- UNIQUE WELL ID'},
  {'_type': 'HeaderItem',
   'mnemonic': 'ENTR',
   'unit': '',
   'value': 'DW',
   'descr': 'Entered - ENTERED BY'},
  {'_type': 'HeaderItem',
   'mnemonic': 'SRVC',
   'unit': '',
   'value': 'COMPUTALOG',
   'd

In [42]:
las2_json_string_cleanedB_2

'{"Version": [{"_type": "HeaderItem", "mnemonic": "VERS", "unit": "", "value": 2.0, "descr": "CWLS LOG ASCII STANDARD -VERSION 2.0"}, {"_type": "HeaderItem", "mnemonic": "WRAP", "unit": "", "value": "NO", "descr": "ONE LINE PER DEPTH STEP"}], "Well": [{"_type": "HeaderItem", "mnemonic": "NULL", "unit": "", "value": -999.25, "descr": "NULL_VALUE"}, {"_type": "HeaderItem", "mnemonic": "WELL", "unit": "", "value": "PETROREP ET AL HOUSE", "descr": "Well_name    - WELL"}, {"_type": "HeaderItem", "mnemonic": "LOC", "unit": "", "value": "01-03-085-15W4", "descr": "Location- LOCATION"}, {"_type": "HeaderItem", "mnemonic": "UWI", "unit": "", "value": "00/01-03-085-15W400", "descr": "Uwi- UNIQUE WELL ID"}, {"_type": "HeaderItem", "mnemonic": "ENTR", "unit": "", "value": "DW", "descr": "Entered - ENTERED BY"}, {"_type": "HeaderItem", "mnemonic": "SRVC", "unit": "", "value": "COMPUTALOG", "descr": "Scn- SERVICE COMPANY"}, {"_type": "HeaderItem", "mnemonic": "DATE", "unit": "", "value": "11 JAN 85"

In [43]:
las_by_hand_copy = [
    {"Version": [
        {"_type": "HeaderItem", "mnemonic": "VERS", "unit": "", "value": 2.0, "descr": "CWLS LOG ASCII STANDARD -VERSION 2.0"}, 
        {"_type": "HeaderItem", "mnemonic": "WRAP", "unit": "", "value": "NO", "descr": "ONE LINE PER DEPTH STEP"}
    ], 
     "Well": 
     [
         {"_type": "HeaderItem", "mnemonic": "NULL", "unit": "", "value": -999.25, "descr": "NULL_VALUE"}, 
         {"_type": "HeaderItem", "mnemonic": "WELL", "unit": "", "value": "PETROREP ET AL HOUSE", "descr": "Well_name    - WELL"}, 
         {"_type": "HeaderItem", "mnemonic": "LOC", "unit": "", "value": "01-03-085-15W4", "descr": "Location- LOCATION"}, 
         {"_type": "HeaderItem", "mnemonic": "UWI", "unit": "", "value": "00/01-03-085-15W400", "descr": "Uwi- UNIQUE WELL ID"},
         {"_type": "HeaderItem", "mnemonic": "ENTR", "unit": "", "value": "DW", "descr": "Entered - ENTERED BY"}, 
         {"_type": "HeaderItem", "mnemonic": "SRVC", "unit": "", "value": "COMPUTALOG", "descr": "Scn- SERVICE COMPANY"}, 
         {"_type": "HeaderItem", "mnemonic": "DATE", "unit": "", "value": "11 JAN 85", "descr": "Date    - LOG DATE"}, 
         {"_type": "HeaderItem", "mnemonic": "STRT", "unit": "M", "value": 97, "descr": "top_depth    - START DEPTH"}, 
         {"_type": "HeaderItem", "mnemonic": "STOP", "unit": "M", "value": 252.102, "descr": "bot_depth    - STOP DEPTH"}, 
         {"_type": "HeaderItem", "mnemonic": "STEP", "unit": "M", "value": 0.3, "descr": "increment    - STEP LENGTH"}
     ],
     "Curves": [
         {"_type": "CurveItem", "mnemonic": "DEPT", "unit": "M", "value": "00 001 00 00", "descr": "DEPTH   - DEPTH", "data": [97.0, 97.3, 97.6, 97.9, 98.2, 98.5, 98.8, 99.1, 99.4, 99.7, 100.0, 100.3, 100.6, 100.9, 101.2, 101.5, 101.8, 102.1, 102.4, 102.7, 103.0, 103.3, 103.6, 103.9, 104.2, 104.5, 104.8, 105.1, 105.4, 105.7, 106.0, 106.3, 106.6, 106.9, 107.2, 107.5, 107.8, 108.1, 108.4, 108.7, 109.0, 109.3, 109.6, 109.9, 110.2, 110.5, 110.8, 111.1, 111.4, 111.7, 112.0, 112.3, 112.6, 112.9, 113.2, 113.5, 113.8, 114.1, 114.4, 114.7, 115.0, 115.3, 115.6, 115.9, 116.2, 116.5, 116.8, 117.1, 117.4, 117.7, 118.0, 118.3, 118.6, 118.9, 119.2, 119.5, 119.8, 120.1, 120.4, 120.7, 121.0, 121.3, 121.6, 121.9, 122.2, 122.5, 122.8, 123.1, 123.4, 123.7, 124.0, 124.3, 124.6, 124.9, 125.2, 125.5, 125.8, 126.1, 126.4, 126.7, 127.0, 127.3, 127.6, 127.9, 128.2, 128.5, 128.8, 129.1, 129.4, 129.7, 130.0, 130.3, 130.6, 130.9, 131.2, 131.5, 131.8, 132.1, 132.4, 132.7, 133.0, 133.3, 133.6, 133.9, 134.2, 134.5, 134.8, 135.1, 135.4, 135.7, 136.0, 136.3, 136.6, 136.9, 137.2, 137.5, 137.8, 138.1, 138.4, 138.7, 139.0, 139.3, 139.6, 139.9, 140.2, 140.5, 140.8, 141.1, 141.4, 141.7, 142.0, 142.3, 142.6, 142.9, 143.2, 143.5, 143.8, 144.1, 144.4, 144.7, 145.0, 145.3, 145.6, 145.9, 146.201, 146.501, 146.801, 147.101, 147.401, 147.701, 148.001, 148.301, 148.601, 148.901, 149.201, 149.501, 149.801, 150.101, 150.401, 150.701, 151.001, 151.301, 151.601, 151.901, 152.201, 152.501, 152.801, 153.101, 153.401, 153.701, 154.001, 154.301, 154.601, 154.901, 155.201, 155.501, 155.801, 156.101, 156.401, 156.701, 157.001, 157.301, 157.601, 157.901, 158.201, 158.501, 158.801, 159.101, 159.401, 159.701, 160.001, 160.301, 160.601, 160.901, 161.201, 161.501, 161.801, 162.101, 162.401, 162.701, 163.001, 163.301, 163.601, 163.901, 164.201, 164.501, 164.801, 165.101, 165.401, 165.701, 166.001, 166.301, 166.601, 166.901, 167.201, 167.501, 167.801, 168.101, 168.401, 168.701, 169.001, 169.301, 169.601, 169.901, 170.201, 170.501, 170.801, 171.101, 171.401, 171.701, 172.001, 172.301, 172.601, 172.901, 173.201, 173.501, 173.801, 174.101, 174.401, 174.701, 175.001, 175.301, 175.601, 175.901, 176.201, 176.501, 176.801, 177.101, 177.401, 177.701, 178.001, 178.301, 178.601, 178.901, 179.201, 179.501, 179.801, 180.101, 180.401, 180.701, 181.001, 181.301, 181.601, 181.901, 182.201, 182.501, 182.801, 183.101, 183.401, 183.701, 184.001, 184.301, 184.601, 184.901, 185.201, 185.501, 185.801, 186.101, 186.401, 186.701, 187.001, 187.301, 187.601, 187.901, 188.201, 188.501, 188.801, 189.101, 189.401, 189.701, 190.001, 190.301, 190.601, 190.901, 191.201, 191.501, 191.801, 192.101, 192.401, 192.701, 193.001, 193.301, 193.601, 193.901, 194.201, 194.501, 194.801, 195.101, 195.401, 195.701, 196.001, 196.301, 196.601, 196.901, 197.201, 197.501, 197.801, 198.101, 198.401, 198.701, 199.001, 199.301, 199.601, 199.901, 200.201, 200.501, 200.801, 201.101, 201.401, 201.701, 202.001, 202.301, 202.601, 202.901, 203.201, 203.501, 203.801, 204.101, 204.401, 204.701, 205.001, 205.301, 205.601, 205.901, 206.201, 206.501, 206.801, 207.101, 207.401, 207.701, 208.001, 208.301, 208.601, 208.901, 209.201, 209.501, 209.801, 210.101, 210.401, 210.701, 211.001, 211.301, 211.601, 211.901, 212.201, 212.501, 212.801, 213.101, 213.401, 213.701, 214.001, 214.301, 214.601, 214.901, 215.201, 215.501, 215.801, 216.101, 216.401, 216.701, 217.001, 217.301, 217.601, 217.901, 218.201, 218.501, 218.801, 219.101, 219.401, 219.701, 220.001, 220.301, 220.601, 220.901, 221.201, 221.501, 221.801, 222.101, 222.401, 222.701, 223.001, 223.301, 223.601, 223.901, 224.201, 224.501, 224.801, 225.101, 225.401, 225.701, 226.001, 226.301, 226.601, 226.901, 227.201, 227.501, 227.801, 228.101, 228.401, 228.701, 229.001, 229.301, 229.601, 229.901, 230.201, 230.501, 230.801, 231.101, 231.401, 231.701, 232.001, 232.301, 232.601, 232.901, 233.201, 233.501, 233.801, 234.101, 234.401, 234.701, 235.001, 235.301, 235.601, 235.901, 236.201, 236.501, 236.801, 237.101, 237.401, 237.701, 238.001, 238.301, 238.601, 238.901, 239.201, 239.501, 239.801, 240.101, 240.401, 240.701, 241.001, 241.301, 241.601, 241.901, 242.201, 242.501, 242.801, 243.101, 243.401, 243.701, 244.001, 244.301, 244.602, 244.902, 245.202, 245.502, 245.802, 246.102, 246.402, 246.702, 247.002, 247.302, 247.602, 247.902, 248.202, 248.502, 248.802, 249.102, 249.402, 249.702, 250.002, 250.302, 250.602, 250.902, 251.202, 251.502, 251.802, 252.102]}, {"_type": "CurveItem", "mnemonic": "DPHI", "unit": "V/V", "value": "00 890 00 00", "descr": "PHID    - DENSITY POROSITY (LIMESTONE)", "data": [0.326, 0.321, 0.289, 0.258, 0.274, 0.303, 0.318, 0.326, 0.322, 0.292, 0.243, 0.21, 0.192, 0.22, 0.241, 0.209, 0.187, 0.205, 0.225, 0.206, 0.19, 0.209, 0.238, 0.254, 0.28, 0.325, 0.33, 0.299, 0.276, 0.267, 0.251, 0.23, 0.227, 0.235, 0.29, 0.43, 0.47, 0.398, 0.338, 0.354, 0.362, 0.358, 0.357, 0.349, 0.344, 0.338, 0.333, 0.329, 0.33, 0.343, 0.354, 0.347, 0.342, 0.343, 0.358, 0.367, 0.355, 0.332, 0.347, 0.358, 0.347, 0.343, 0.35, 0.357, 0.358, 0.353, 0.337, 0.332, 0.347, 0.355, 0.351, 0.361, 0.348, 0.347, 0.361, 0.359, 0.354, 0.347, 0.345, 0.361, 0.359, 0.333, 0.22, 0.042, 0.066, 0.173, 0.345, 0.366, 0.369, 0.352, 0.336, 0.337, 0.349, 0.349, 0.347, 0.35, 0.355, 0.355, 0.351, 0.332, 0.34, 0.344, 0.338, 0.344, 0.349, 0.346, 0.327, 0.301, 0.305, 0.319, 0.307, 0.31, 0.355, 0.344, 0.305, 0.296, 0.283, 0.278, 0.276, 0.271, 0.265, 0.259, 0.252, 0.248, 0.244, 0.244, 0.249, 0.267, 0.267, 0.255, 0.261, 0.276, 0.287, 0.287, 0.282, 0.266, 0.261, 0.267, 0.28, 0.293, 0.309, 0.322, 0.339, 0.332, 0.291, 0.3, 0.278, 0.106, 0.0, 0.099, 0.189, 0.26, 0.28, 0.262, 0.259, 0.259, 0.257, 0.259, 0.271, 0.276, 0.261, 0.26, 0.266, 0.27, 0.271, 0.279, 0.269, 0.254, 0.256, 0.275, 0.255, 0.241, 0.235, 0.251, 0.287, 0.303, 0.307, 0.303, 0.294, 0.286, 0.278, 0.272, 0.274, 0.279, 0.272, 0.271, 0.278, 0.272, 0.267, 0.276, 0.271, 0.262, 0.259, 0.255, 0.267, 0.259, 0.262, 0.283, 0.301, 0.287, 0.266, 0.243, 0.234, 0.326, 0.34, 0.326, 0.288, 0.27, 0.261, 0.249, 0.261, 0.262, 0.258, 0.258, 0.263, 0.266, 0.247, 0.244, 0.248, 0.246, 0.214, 0.202, 0.256, 0.285, 0.279, 0.253, 0.242, 0.231, 0.214, 0.237, 0.262, 0.279, 0.308, 0.31, 0.238, 0.151, 0.242, 0.285, 0.302, 0.307, 0.312, 0.314, 0.312, 0.304, 0.283, 0.265, 0.266, 0.266, 0.264, 0.26, 0.26, 0.269, 0.277, 0.265, 0.261, 0.272, 0.275, 0.27, 0.263, 0.301, 0.292, 0.248, 0.299, 0.34, 0.317, 0.299, 0.317, 0.329, 0.334, 0.321, 0.24, 0.076, 0.039, 0.037, 0.052, 0.181, 0.317, 0.324, 0.326, 0.322, 0.223, 0.178, 0.254, 0.315, 0.322, 0.319, 0.304, 0.28, 0.269, 0.286, 0.288, 0.277, 0.287, 0.307, 0.311, 0.311, 0.304, 0.286, 0.265, 0.263, 0.262, 0.257, 0.256, 0.246, 0.222, 0.215, 0.243, 0.26, 0.252, 0.237, 0.246, 0.236, 0.227, 0.231, 0.252, 0.285, 0.317, 0.332, 0.334, 0.315, 0.29, 0.272, 0.265, 0.284, 0.305, 0.299, 0.26, 0.259, 0.271, 0.254, 0.239, 0.288, 0.294, 0.268, 0.242, 0.275, 0.297, 0.298, 0.266, 0.254, 0.277, 0.284, 0.269, 0.245, 0.245, 0.227, 0.236, 0.24, 0.235, 0.229, 0.235, 0.23, 0.224, 0.216, 0.222, 0.233, 0.235, 0.234, 0.231, 0.225, 0.191, 0.199, 0.215, 0.22, 0.22, 0.215, 0.205, 0.197, 0.22, 0.228, 0.225, 0.223, 0.221, 0.223, 0.223, 0.221, 0.223, 0.234, 0.247, 0.24, 0.239, 0.259, 0.275, 0.291, 0.281, 0.253, 0.256, 0.271, 0.268, 0.234, 0.229, 0.291, 0.287, 0.258, 0.272, 0.29, 0.277, 0.277, 0.28, 0.262, 0.228, 0.237, 0.248, 0.252, 0.25, 0.24, 0.245, 0.243, 0.237, 0.23, 0.233, 0.24, 0.231, 0.223, 0.222, 0.225, 0.23, 0.24, 0.265, 0.265, 0.26, 0.242, 0.228, 0.227, 0.217, 0.178, 0.192, 0.221, 0.213, 0.179, 0.167, 0.206, 0.23, 0.249, 0.311, 0.315, 0.306, 0.304, 0.302, 0.298, 0.299, 0.292, 0.291, 0.3, 0.293, 0.272, 0.269, 0.273, 0.284, 0.29, 0.286, 0.239, 0.219, 0.21, 0.231, 0.248, 0.256, 0.25, 0.239, 0.232, 0.239, 0.243, 0.21, 0.182, 0.228, 0.243, 0.252, 0.252, 0.228, 0.205, 0.214, 0.271, 0.305, 0.339, 0.348, 0.325, 0.299, 0.27, 0.257, 0.25, 0.246, 0.295, 0.294, 0.286, 0.279, 0.264, 0.245, 0.229, 0.222, 0.218, 0.237, 0.276, 0.288, 0.293, 0.281, 0.269, 0.267, 0.247, 0.136, 0.102, 0.113, 0.141, 0.159, 0.167, 0.15, 0.126, 0.119, 0.12, 0.131, 0.144, 0.146, 0.145, 0.151, 0.16, 0.163, 0.154, 0.155, 0.162]}, {"_type": "CurveItem", "mnemonic": "NPHI", "unit": "V/V", "value": "00 330 00 00", "descr": "PHIN    - NEUTRON POROSITY (LIMESTONE)", "data": [0.412, 0.384, 0.374, 0.398, 0.407, 0.416, 0.416, 0.387, 0.35, 0.351, 0.412, 0.446, 0.459, 0.478, 0.471, 0.455, 0.452, 0.454, 0.461, 0.478, 0.475, 0.47, 0.507, 0.483, 0.418, 0.401, 0.408, 0.411, 0.397, 0.395, 0.414, 0.411, 0.403, 0.4, 0.41, 0.505, 0.507, 0.417, 0.381, 0.358, 0.275, 0.262, 0.27, 0.303, 0.339, 0.367, 0.39, 0.409, 0.412, 0.405, 0.404, 0.405, 0.415, 0.41, 0.399, 0.395, 0.394, 0.4, 0.41, 0.413, 0.409, 0.404, 0.368, 0.347, 0.377, 0.417, 0.439, 0.423, 0.393, 0.393, 0.422, 0.414, 0.41, 0.407, 0.408, 0.41, 0.419, 0.429, 0.431, 0.422, 0.394, 0.296, 0.22, 0.165, 0.176, 0.254, 0.309, 0.398, 0.413, 0.389, 0.418, 0.437, 0.426, 0.385, 0.397, 0.421, 0.407, 0.382, 0.378, 0.403, 0.408, 0.406, 0.403, 0.412, 0.412, 0.427, 0.445, 0.44, 0.412, 0.418, 0.424, 0.408, 0.434, 0.45, 0.45, 0.437, 0.395, 0.387, 0.395, 0.446, 0.478, 0.47, 0.425, 0.417, 0.457, 0.429, 0.44, 0.435, 0.429, 0.428, 0.435, 0.465, 0.462, 0.42, 0.41, 0.457, 0.473, 0.461, 0.453, 0.437, 0.426, 0.419, 0.408, 0.377, 0.381, 0.419, 0.399, 0.219, 0.163, 0.158, 0.225, 0.321, 0.377, 0.428, 0.432, 0.432, 0.423, 0.42, 0.439, 0.451, 0.456, 0.452, 0.429, 0.408, 0.408, 0.446, 0.429, 0.413, 0.421, 0.452, 0.429, 0.446, 0.453, 0.424, 0.408, 0.399, 0.393, 0.395, 0.418, 0.432, 0.428, 0.442, 0.44, 0.427, 0.425, 0.45, 0.497, 0.468, 0.431, 0.45, 0.474, 0.487, 0.475, 0.426, 0.413, 0.451, 0.464, 0.446, 0.422, 0.453, 0.453, 0.436, 0.431, 0.432, 0.424, 0.412, 0.414, 0.43, 0.435, 0.415, 0.397, 0.387, 0.463, 0.473, 0.46, 0.439, 0.428, 0.419, 0.415, 0.429, 0.415, 0.401, 0.44, 0.416, 0.397, 0.414, 0.432, 0.418, 0.42, 0.469, 0.475, 0.464, 0.442, 0.4, 0.339, 0.341, 0.393, 0.425, 0.441, 0.439, 0.422, 0.418, 0.419, 0.43, 0.448, 0.441, 0.42, 0.417, 0.467, 0.486, 0.475, 0.432, 0.448, 0.456, 0.456, 0.451, 0.445, 0.44, 0.417, 0.392, 0.395, 0.398, 0.375, 0.36, 0.359, 0.399, 0.467, 0.46, 0.43, 0.382, 0.342, 0.229, 0.195, 0.19, 0.265, 0.326, 0.375, 0.444, 0.44, 0.418, 0.368, 0.357, 0.381, 0.403, 0.418, 0.427, 0.398, 0.389, 0.419, 0.434, 0.426, 0.432, 0.454, 0.45, 0.43, 0.434, 0.435, 0.427, 0.416, 0.411, 0.438, 0.443, 0.414, 0.383, 0.409, 0.435, 0.442, 0.457, 0.467, 0.449, 0.455, 0.442, 0.429, 0.43, 0.439, 0.418, 0.352, 0.312, 0.308, 0.361, 0.398, 0.411, 0.418, 0.422, 0.418, 0.413, 0.414, 0.406, 0.39, 0.384, 0.394, 0.389, 0.383, 0.383, 0.391, 0.399, 0.399, 0.395, 0.409, 0.415, 0.386, 0.386, 0.412, 0.421, 0.427, 0.44, 0.45, 0.442, 0.431, 0.423, 0.417, 0.414, 0.417, 0.447, 0.462, 0.464, 0.457, 0.437, 0.422, 0.428, 0.415, 0.428, 0.456, 0.459, 0.432, 0.418, 0.406, 0.405, 0.394, 0.369, 0.36, 0.398, 0.396, 0.381, 0.389, 0.403, 0.407, 0.411, 0.417, 0.401, 0.364, 0.395, 0.402, 0.377, 0.367, 0.363, 0.362, 0.402, 0.434, 0.43, 0.409, 0.397, 0.397, 0.434, 0.431, 0.409, 0.399, 0.394, 0.395, 0.4, 0.396, 0.396, 0.415, 0.414, 0.403, 0.39, 0.384, 0.389, 0.429, 0.425, 0.406, 0.398, 0.41, 0.417, 0.424, 0.439, 0.461, 0.479, 0.49, 0.475, 0.465, 0.453, 0.438, 0.426, 0.408, 0.393, 0.383, 0.386, 0.404, 0.405, 0.397, 0.38, 0.37, 0.366, 0.349, 0.306, 0.312, 0.368, 0.385, 0.397, 0.394, 0.403, 0.356, 0.349, 0.394, 0.392, 0.383, 0.38, 0.403, 0.398, 0.383, 0.387, 0.426, 0.432, 0.427, 0.477, 0.502, 0.468, 0.441, 0.448, 0.446, 0.439, 0.435, 0.432, 0.488, 0.47, 0.465, 0.459, 0.45, 0.417, 0.39, 0.367, 0.347, 0.339, 0.349, 0.366, 0.391, 0.45, 0.453, 0.428, 0.371, 0.361, 0.368, 0.397, 0.413, 0.426, 0.434, 0.418, 0.396, 0.366, 0.343, 0.338, 0.36, 0.371, 0.372, 0.366, 0.367, 0.37, 0.366, 0.375, 0.436, 0.433, 0.432, 0.444, 0.423, 0.399, 0.389, 0.395, 0.403, 0.407, 0.402, 0.39, 0.386, 0.389, 0.384, 0.373, 0.369, 0.371]}, {"_type": "CurveItem", "mnemonic": "GR", "unit": "API", "value": "00 310 00 00", "descr": "GR - GAMMA RAY", "data": [91.436, 103.41, 116.013, 116.635, 83.204, 69.322, 61.116, 54.8, 69.295, 91.358, 112.159, 122.239, 126.014, 128.528, 133.562, 135.448, 126.612, 120.928, 122.178, 121.539, 130.36, 136.027, 134.126, 117.091, 92.49, 76.715, 76.707, 87.417, 94.976, 101.275, 110.724, 111.978, 113.858, 108.807, 96.186, 74.107, 64.008, 72.828, 77.23, 76.593, 71.538, 67.747, 65.216, 63.317, 67.721, 65.822, 60.768, 62.648, 63.904, 63.895, 60.733, 61.987, 62.607, 67.645, 80.878, 63.212, 62.573, 60.672, 56.25, 61.915, 65.692, 65.683, 62.521, 57.464, 55.568, 55.556, 58.07, 60.583, 60.575, 59.308, 56.144, 56.138, 60.541, 59.272, 56.742, 59.255, 49.788, 55.455, 60.491, 52.915, 56.689, 58.573, 51.626, 45.312, 45.935, 59.167, 62.941, 62.304, 58.512, 56.609, 57.859, 59.114, 60.369, 57.837, 56.568, 62.864, 55.918, 54.651, 60.946, 66.612, 68.496, 67.227, 62.804, 62.793, 64.046, 64.668, 66.551, 67.174, 67.166, 67.157, 73.454, 76.597, 77.852, 78.472, 75.942, 79.718, 81.598, 76.547, 82.215, 87.251, 92.287, 96.061, 96.052, 87.215, 83.424, 85.306, 91.601, 85.288, 79.605, 85.899, 85.263, 78.947, 80.832, 84.604, 85.228, 87.742, 91.516, 92.139, 86.453, 89.598, 87.697, 80.123, 75.7, 61.818, 64.962, 85.763, 84.494, 58.0, 51.685, 51.045, 58.605, 75.622, 95.161, 93.889, 90.096, 88.198, 86.929, 91.966, 86.28, 85.64, 91.939, 96.344, 96.967, 89.391, 87.49, 84.957, 84.95, 88.725, 92.5, 97.534, 97.526, 96.887, 94.358, 79.215, 59.657, 57.126, 61.532, 71.612, 74.124, 76.009, 83.567, 93.649, 87.334, 82.912, 81.01, 79.11, 85.407, 91.704, 86.02, 89.164, 93.568, 93.562, 89.139, 85.346, 87.229, 91.633, 88.474, 74.592, 71.427, 77.097, 95.376, 99.15, 77.701, 77.059, 79.575, 84.61, 90.278, 97.837, 95.307, 93.406, 97.809, 99.063, 99.055, 96.522, 93.359, 92.722, 94.604, 98.38, 99.633, 94.58, 90.788, 93.3, 90.768, 82.562, 81.294, 86.33, 85.691, 71.808, 74.953, 95.124, 98.897, 95.736, 89.423, 78.063, 65.442, 85.613, 94.432, 91.901, 78.65, 74.858, 81.154, 83.037, 84.921, 88.696, 91.209, 89.94, 84.886, 89.29, 91.806, 90.534, 89.895, 92.41, 100.599, 103.113, 92.384, 87.33, 89.844, 87.942, 94.24, 97.386, 92.963, 91.063, 92.945, 93.567, 91.036, 81.567, 84.081, 87.225, 85.957, 69.553, 61.975, 65.751, 67.002, 65.735, 67.616, 70.762, 75.167, 78.31, 80.195, 76.403, 61.259, 67.555, 79.529, 79.519, 76.359, 76.349, 78.862, 79.484, 81.367, 82.618, 83.241, 83.234, 80.072, 76.281, 74.378, 75.632, 81.299, 85.704, 86.325, 86.948, 91.353, 98.282, 98.903, 93.848, 91.947, 92.568, 93.195, 95.707, 96.328, 90.645, 90.006, 95.671, 98.184, 96.916, 89.972, 79.242, 83.646, 86.159, 85.522, 81.729, 80.46, 81.081, 76.028, 74.128, 77.903, 82.307, 84.823, 82.921, 79.129, 84.165, 86.677, 87.931, 92.968, 94.221, 84.752, 80.33, 84.735, 91.033, 91.022, 87.232, 84.699, 84.691, 86.575, 89.719, 85.297, 93.485, 101.675, 110.495, 114.901, 102.909, 94.074, 85.866, 92.795, 103.507, 106.018, 104.116, 106.632, 107.885, 107.245, 112.28, 112.274, 111.635, 114.149, 117.292, 110.978, 104.662, 102.763, 100.231, 100.223, 107.15, 108.404, 107.134, 105.863, 105.224, 102.061, 107.728, 110.874, 102.665, 99.504, 97.604, 96.964, 96.326, 91.27, 88.108, 86.209, 86.832, 93.131, 89.966, 88.699, 86.797, 84.896, 82.366, 78.573, 76.039, 75.402, 77.287, 82.953, 82.312, 79.783, 82.296, 87.331, 89.216, 90.467, 93.611, 96.126, 98.01, 99.262, 98.624, 93.569, 91.039, 94.183, 92.913, 89.118, 87.219, 85.951, 97.292, 106.113, 107.365, 103.571, 108.608, 115.536, 121.833, 122.455, 112.358, 96.583, 89.639, 97.826, 103.494, 99.07, 94.648, 94.008, 94.001, 82.011, 70.021, 56.769, 49.824, 45.401, 49.805, 52.319, 52.31, 51.039, 51.663, 52.916, 53.538, 58.574, 67.393, 69.277, 67.373, 66.736, 71.142, 86.269, 96.349, 91.296, 87.504, 98.846, 113.341, 108.285, 104.494, 104.487, 111.415, 115.82, 109.505, 98.146, 91.831, 104.435, 115.776, 117.659, 115.758, 112.598, 104.391, 89.247, 75.366, 57.7, 51.385, 50.114, 55.152, 71.539, 89.187, 92.331, 76.558, 62.675, 61.405, 67.703, 81.568, 89.757, 95.424, 101.09, 108.648, 113.053, 113.045, 89.705, 68.885, 70.138, 69.5, 66.337, 66.33, 71.996, 94.059, 122.427, 126.83, 125.56, 124.922, 126.174, 128.059, 130.569, 130.563, 128.031, 126.763, 128.643, 129.897, 130.519, 131.142, 129.87, 127.341, 123.548, 92.641, 52.273, 25.149]}, {"_type": "CurveItem", "mnemonic": "ILD", "unit": "OHMM", "value": "00 120 00 00", "descr": "RESD    - DEEP RESISTIVITY (DIL)", "data": [16.597, 15.074, 13.957, 13.174, 14.785, 16.916, 18.624, 18.624, 16.915, 12.432, 9.496, 7.684, 6.716, 6.218, 5.757, 5.757, 5.983, 6.099, 5.868, 5.228, 4.482, 4.15, 4.313, 6.585, 10.86, 14.492, 15.955, 14.773, 11.954, 10.858, 10.248, 9.673, 9.13, 9.307, 10.051, 12.662, 14.768, 18.249, 22.549, 26.812, 31.88, 32.498, 29.516, 25.796, 23.884, 23.428, 22.542, 22.112, 21.276, 20.472, 20.081, 20.081, 20.47, 20.079, 18.953, 16.885, 16.884, 17.546, 18.234, 19.692, 20.464, 21.266, 22.529, 22.966, 22.965, 22.097, 21.675, 20.459, 19.686, 18.942, 18.58, 18.94, 18.94, 20.064, 20.454, 20.453, 20.452, 19.304, 18.221, 18.22, 18.573, 22.513, 47.679, 45.003, 37.125, 25.756, 20.445, 17.868, 16.865, 16.227, 16.863, 17.525, 17.524, 17.189, 16.224, 15.314, 14.454, 14.454, 15.312, 16.222, 15.912, 15.019, 14.451, 15.018, 15.309, 14.174, 12.628, 11.92, 11.919, 11.25, 10.619, 9.644, 8.759, 8.759, 9.46, 10.414, 10.823, 10.414, 10.02, 8.427, 6.561, 5.959, 5.846, 5.959, 6.56, 7.085, 7.363, 7.363, 7.505, 7.799, 7.951, 8.105, 7.95, 7.65, 6.948, 6.31, 6.19, 6.072, 6.309, 7.359, 10.405, 14.997, 21.203, 22.898, 20.797, 17.49, 16.829, 19.254, 23.792, 28.29, 30.552, 25.693, 16.825, 12.606, 10.201, 9.265, 9.445, 10.2, 10.6, 11.015, 11.015, 11.015, 10.804, 10.598, 10.598, 10.395, 10.803, 11.012, 11.012, 10.801, 10.393, 9.623, 8.91, 8.74, 9.999, 13.868, 14.412, 11.889, 9.998, 7.936, 7.348, 6.179, 5.505, 5.505, 5.946, 6.546, 6.802, 7.069, 6.934, 6.802, 6.297, 6.059, 5.944, 5.83, 5.719, 5.61, 5.719, 6.176, 6.67, 6.931, 6.295, 5.502, 6.417, 8.083, 10.997, 13.589, 14.396, 12.106, 10.786, 9.796, 9.246, 9.246, 9.425, 9.245, 9.069, 8.896, 8.726, 8.726, 8.396, 7.773, 7.773, 8.078, 8.557, 8.557, 8.077, 8.234, 9.42, 10.371, 10.173, 8.555, 6.791, 5.935, 6.05, 6.534, 8.074, 10.171, 8.719, 7.919, 8.39, 9.599, 10.169, 10.366, 9.974, 9.414, 9.235, 8.886, 8.716, 8.55, 8.07, 7.765, 7.472, 7.471, 7.328, 7.052, 6.785, 6.917, 7.763, 8.713, 9.778, 10.359, 9.591, 9.053, 9.229, 10.358, 12.081, 13.3, 13.558, 13.299, 13.557, 14.926, 17.747, 20.7, 28.161, 41.378, 36.864, 27.095, 19.535, 14.084, 11.397, 10.154, 10.154, 10.552, 11.396, 11.842, 12.072, 12.071, 12.071, 11.615, 11.614, 11.175, 10.548, 10.548, 11.174, 11.612, 12.067, 12.54, 12.784, 12.539, 11.835, 10.75, 9.216, 8.533, 6.905, 5.174, 5.376, 5.919, 6.904, 7.748, 8.052, 8.052, 8.051, 8.208, 8.367, 8.695, 9.572, 10.539, 12.058, 15.189, 17.047, 17.046, 15.187, 13.531, 12.528, 11.825, 11.824, 12.77, 13.017, 12.287, 11.822, 11.822, 12.051, 13.015, 13.267, 12.766, 11.374, 11.819, 12.764, 13.012, 12.764, 12.281, 12.519, 12.519, 12.518, 12.046, 11.153, 10.731, 9.025, 6.142, 6.261, 7.444, 9.745, 11.367, 10.729, 8.85, 7.885, 6.891, 6.631, 6.891, 7.161, 7.441, 7.441, 7.441, 7.733, 8.351, 9.372, 11.361, 12.035, 13.508, 13.77, 13.506, 13.249, 12.748, 12.266, 11.803, 10.72, 9.925, 10.118, 10.927, 12.03, 13.244, 13.244, 13.763, 14.302, 14.579, 14.579, 14.301, 14.028, 13.24, 12.259, 12.025, 11.795, 11.795, 12.024, 12.023, 12.495, 12.984, 12.984, 12.736, 12.255, 11.13, 10.109, 9.727, 10.108, 10.708, 11.789, 12.251, 12.251, 12.251, 12.017, 11.342, 10.914, 9.723, 8.663, 8.177, 7.718, 7.146, 5.895, 4.77, 3.86, 3.786, 3.935, 4.678, 6.008, 8.332, 12.009, 18.694, 21.389, 21.388, 19.425, 16.977, 14.277, 12.239, 10.905, 10.493, 14.003, 20.182, 25.423, 32.648, 36.641, 37.352, 35.94, 33.277, 28.528, 23.534, 18.325, 14.546, 13.73, 14.828, 15.115, 12.711, 8.99, 7.56, 6.607, 5.556, 4.584, 3.93, 3.781, 3.929, 4.244, 4.326, 4.326, 4.325, 4.243, 4.243, 4.082, 3.928, 3.928, 4.242, 5.448, 7.555, 10.279, 14.815, 20.546, 26.385, 27.952, 20.152, 10.081, 5.552, 5.446, 7.408, 8.641, 12.454, 13.45, 6.6, 5.994, 5.88, 6.229, 7.127, 8.977, 11.528, 13.974, 16.938, 22.174, 28.474, 34.515, 37.275, 33.854, 29.023, 15.087, 10.268, 8.802, 7.842, 7.842, 8.633, 8.972, 9.324, 9.689, 9.689, 9.504, 9.145, 8.799, 8.631, 8.305, 8.146, 8.146, 8.146, 8.304, 8.145]}]", "Parameter": [{"_type": "HeaderItem", "mnemonic": "GL", "unit": "M", "value": 528.69, "descr": "gl - GROUND LEVEL ELEVATION"}, {"_type": "HeaderItem", "mnemonic": "EREF", "unit": "M", "value": 532.68, "descr": "kb - ELEVATION OF DEPTH REFERENCE"}, {"_type": "HeaderItem", "mnemonic": "DATM", "unit": "M", "value": 532.68, "descr": "datum   - DATUM ELEVATION"}, {"_type": "HeaderItem", "mnemonic": "TDD", "unit": "M", "value": 252, "descr": "tdd- TOTAL DEPTH DRILLER"}, {"_type": "HeaderItem", "mnemonic": "RUN", "unit": "", "value": "ONE", "descr": "Run- RUN NUMBER"}, {"_type": "HeaderItem", "mnemonic": "ENG", "unit": "", "value": "BILL", "descr": "Engineer- RECORDING ENGINEER"}, {"_type": "HeaderItem", "mnemonic": "WIT", "unit": "", "value": "JIM", "descr": "Witness - WITNESSED BY"}, {"_type": "HeaderItem", "mnemonic": "MUD", "unit": "", "value": "GEL", "descr": "Mud_type- MUD TYPE"}, {"_type": "HeaderItem", "mnemonic": "MATR", "unit": "", "value": "LIMESTONE", "descr": "Logunit - NEUTRON MATRIX"}, {"_type": "HeaderItem", "mnemonic": "TMAX", "unit": "C", "value": 20, "descr": "BHT- MAXIMUM RECORDED TEMPERATURE"}, {"_type": "HeaderItem", "mnemonic": "BHTD", "unit": "M", "value": 252, "descr": "BHTDEP  - MAXIMUM RECORDED TEMPERATURE"}, {"_type": "HeaderItem", "mnemonic": "MATR", "unit": "", "value": 1, "descr": "LOGUNIT - NEUTRON MATRIX"}, {"_type": "HeaderItem", "mnemonic": "RMT", "unit": "C", "value": 25, "descr": "MDTP    - TEMPERATURE OF MUD"}, {"_type": "HeaderItem", "mnemonic": "NEUT", "unit": "", "value": 1, "descr": "NEUTRON - NEUTRON TYPE"}, {"_type": "HeaderItem", "mnemonic": "RESI", "unit": "", "value": 0, "descr": "RESIST  - RESISTIVITY TYPE"}, {"_type": "HeaderItem", "mnemonic": "RMC", "unit": "OHMM", "value": 0.25, "descr": "RMC- RESISTIVITY OF MUD CAKE"}, {"_type": "HeaderItem", "mnemonic": "RMF", "unit": "OHMM", "value": 0.25, "descr": "RMF- RESISTIVITY OF MUD FILTRATE"}, {"_type": "HeaderItem", "mnemonic": "SUFT", "unit": "C", "value": 15, "descr": "SUFT    - SURFACE TEMPERATURE"}]", "Other": ""}]

SyntaxError: invalid syntax (<ipython-input-43-237c9abac27f>, line 20)

In [44]:
las2.well

[HeaderItem(mnemonic=NULL, unit=, value=-999.25, descr=NULL_VALUE),
 HeaderItem(mnemonic=WELL, unit=, value=PETROREP ET AL HOUSE, descr=Well_name...),
 HeaderItem(mnemonic=LOC, unit=, value=01-03-085-15W4, descr=Location     - L...),
 HeaderItem(mnemonic=UWI, unit=, value=00/01-03-085-15W400, descr=Uwi        ...),
 HeaderItem(mnemonic=ENTR, unit=, value=DW, descr=Entered      - ENTERED BY),
 HeaderItem(mnemonic=SRVC, unit=, value=COMPUTALOG, descr=Scn          - SERV...),
 HeaderItem(mnemonic=DATE, unit=, value=11 JAN 85, descr=Date         - LOG DATE),
 HeaderItem(mnemonic=STRT, unit=M, value=97, descr=top_depth    - START DEPTH),
 HeaderItem(mnemonic=STOP, unit=M, value=252.102, descr=bot_depth    - STOP D...),
 HeaderItem(mnemonic=STEP, unit=M, value=0.3, descr=increment    - STEP LENGTH)]

In [45]:
json_las2 = json.dumps(las2, cls=lasio.JSONEncoder)
las_json_dict_2 =json.loads(json_las2)

In [46]:
las_json_dict_2

{'metadata': {'Version': [{}, {}],
  'Well': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}],
  'Curves': [{}, {}, {}, {}, {}],
  'Parameter': [{},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {}],
  'Other': ''},
 'data': {'DEPT': [97,
   97.3,
   97.6,
   97.9,
   98.2,
   98.5,
   98.8,
   99.1,
   99.4,
   99.7,
   100,
   100.3,
   100.6,
   100.9,
   101.2,
   101.5,
   101.8,
   102.1,
   102.4,
   102.7,
   103,
   103.3,
   103.6,
   103.9,
   104.2,
   104.5,
   104.8,
   105.1,
   105.4,
   105.7,
   106,
   106.3,
   106.6,
   106.9,
   107.2,
   107.5,
   107.8,
   108.1,
   108.4,
   108.7,
   109,
   109.3,
   109.6,
   109.9,
   110.2,
   110.5,
   110.8,
   111.1,
   111.4,
   111.7,
   112,
   112.3,
   112.6,
   112.9,
   113.2,
   113.5,
   113.8,
   114.1,
   114.4,
   114.7,
   115,
   115.3,
   115.6,
   115.9,
   116.2,
   116.5,
   116.8,
   117.1,
   117.4,
   117.7,
   118,
   118.3,
   118.6,
   

In [47]:
las2_json_dict_cleaned['Well']

[{'_type': 'HeaderItem',
  'mnemonic': 'NULL',
  'unit': '',
  'value': -999.25,
  'descr': 'NULL_VALUE'},
 {'_type': 'HeaderItem',
  'mnemonic': 'WELL',
  'unit': '',
  'value': 'PETROREP ET AL HOUSE',
  'descr': 'Well_name    - WELL'},
 {'_type': 'HeaderItem',
  'mnemonic': 'LOC',
  'unit': '',
  'value': '01-03-085-15W4',
  'descr': 'Location- LOCATION'},
 {'_type': 'HeaderItem',
  'mnemonic': 'UWI',
  'unit': '',
  'value': '00/01-03-085-15W400',
  'descr': 'Uwi- UNIQUE WELL ID'},
 {'_type': 'HeaderItem',
  'mnemonic': 'ENTR',
  'unit': '',
  'value': 'DW',
  'descr': 'Entered - ENTERED BY'},
 {'_type': 'HeaderItem',
  'mnemonic': 'SRVC',
  'unit': '',
  'value': 'COMPUTALOG',
  'descr': 'Scn- SERVICE COMPANY'},
 {'_type': 'HeaderItem',
  'mnemonic': 'DATE',
  'unit': '',
  'value': '11 JAN 85',
  'descr': 'Date    - LOG DATE'},
 {'_type': 'HeaderItem',
  'mnemonic': 'STRT',
  'unit': 'M',
  'value': 97,
  'descr': 'top_depth    - START DEPTH'},
 {'_type': 'HeaderItem',
  'mnemonic

In [49]:
%%node


    var lj3 = las2_json_dict_cleaned
    std_headers = {
      'Version': 'VERSION INFORMATION',
      'Well': 'WELL INFORMATION BLOCK',
      'Curves': 'CURVE INFORMATION BLOCK', 
      'Parameter': 'PARAMETER INFORMATION'
    };

    lasjson = {};
    lasjson["VERSION INFORMATION"] = lj3['Version'];
    lasjson["WELL INFORMATION BLOCK"] = lj3['Well'];
    lasjson["CURVE INFORMATION BLOCK"] = lj3['Curves'];
    lasjson["PARAMETER INFORMATION"] = lj3['Parameter'];
    lasjson["CURVES"] = lj3.data;
    print("metadata",lj3.metadata)
    for (item in lj3.metadata) {
      if ( !(item in std_headers) ) {
        lasjson[item.toUpperCase()] = lj3.metadata[item];
      }
      else {
        for (var mnemonic in lj3.metadata[item]) {
          section = std_headers[item];      
          lasjson[section][mnemonic] = {
            MNEM: mnemonic,
            UNIT: '',
            DATA: lj3.metadata[item][mnemonic],
            'DESCRIPTION OF MNEMONIC 1': '',
            'DESCRIPTION OF MNEMONIC 2': ''
          };
        }
      }
    print(lasjson)
    }

... ... ... ... ...
"metadata"
... ..... ..... ... ..... ....... ....... ......... ......... ......... ......... ......... ......... ....... ..... ... ...


In [48]:
las2_json_dict_cleaned.metadata

AttributeError: 'dict' object has no attribute 'metadata'

In [50]:
lasjson.keys()

dict_keys(['VERSION INFORMATION', 'WELL INFORMATION BLOCK', 'CURVE INFORMATION BLOCK', 'PARAMETER INFORMATION'])

In [ ]:
lasjson['WELL INFORMATION BLOCK']

In [ ]:
lasjson

In [ ]:
%%node
///wellio_las = wellio.loadLAS("../assets/00-01-03-085-15W4-0.LAS")

In [ ]:
%%node
three_things_2 = wellioviz.fromJSONofWEllGetThingsForPlotting(lasjson,"DEPT")

In [ ]:
thing = "a"

In [ ]:
%%node
console.log(thing)

pixiedust_node 0.2.5 started. Cells starting '%%node' may contain Node.js code.
